In [ ]:
import numpy as np
from PIL import Image
from numba import vectorize

im = Image.open('image.tif')
imarray = np.array(im)
imfft = np.fft.rfft2(imarray)
print("The size of the image array is:\n", imarray.shape)
print("The image array is:\n", imarray)
print("The size of the image fft array is:\n", imfft.shape)
print("The image fft array is:\n", imfft)

@vectorize(['float32(float32, float32)'], target='cuda')
def Add(a, b):
  return a + b

# Initialize arrays
N = 100000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on GPU
C = Add(A, B)
